In [409]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import urllib.request as url
import re
 

#### I found a link on ryanair website with all the routes , so I extract the data from that page

In [410]:
columns=['from','to']
ryanair_route = pd.DataFrame( columns=columns)
for page in range(1,5):
    html_page = url.urlopen("https://flights.ryanair.com/en-gb/sitemap/city-to-city-flights/page-"+str(page))
    soup = BeautifulSoup(html_page)
    for link in soup.findAll('a', href=re.compile("flights-from-")):
        #print (link.text)#get('href'))
        ryanair_route=ryanair_route.append({'from':link.text[:link.text.find("-")-1], 'to':link.text[link.text.find("-")+1:]}, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [411]:
ryanair_route=ryanair_route[ryanair_route["from"]!="Flights from citie"]
ryanair_route=ryanair_route[ryanair_route["from"]!="Flights from countrie"]

In [412]:
ryanair_route

,from,to
0,Aalborg,London
1,Aarhus,Dublin
2,Aarhus,Eindhoven
3,Aarhus,Gdansk
4,Aarhus,London
5,Aberdeen,Alicante
6,Aberdeen,Faro
7,Aberdeen,Malaga
8,Agadir,Charleroi
9,Agadir,Dusseldorf


#### Easyjet is a little bit more complicated as the routes are saved in a script in the web page and also the connections are mapped with iata code, so I need to first map the iata code, even if probably the best solution would be to map the ryanair in iata and use only iata code as we can be sure they unique and standard for every airport.

In [136]:
html_page = url.urlopen("https://www.easyjet.com/en/flight-tracker")
soup = BeautifulSoup(html_page)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [208]:
scripts=soup.findAll('script')
columns=['iata','city']
mapping_iata = pd.DataFrame( columns=columns)
for script in scripts:
    if script.text.find("Sitecore_WorldwideSettings")>0:
        script_text=script.text[int(script.text.find("Sitecore_RoutesData")):]
        #print(script)
        iata=[]
        city=[]
       # print()
        while script_text.find('"Name":')>0:
            iata.append(script_text[int(script_text.find('"Name":'))-5:int(script_text.find('"Name":'))-2])
            script_text=script_text[int(script_text.find('"Name":')):]
            city.append(script_text[int(script_text.find('"Name":'))+8:int(script_text.find('"},{"'))])
            script_text=script_text[int(script_text.find('"},{"')):]
        
        #script=script

In [209]:
len(city)

176

In [210]:
len(iata)

176

In [211]:
for i in range(len(iata)):
    mapping_iata=mapping_iata.append({'iata':iata[i],'city':city[i]}, ignore_index=True)

In [212]:
mapping_iata=mapping_iata.iloc[:169]

In [216]:
mapping_iata.iloc[168]['city']='Zurich'

In [274]:
mapping_iata

,iata,city
0,AAR,Aarhus
1,ABZ,Aberdeen
2,AGA,Agadir
3,ALC,Alicante
4,LEI,Almeria
5,AMS,Amsterdam
6,AOI,Ancona
7,AYT,Antalya
8,AQJ,Aqaba
9,OSD,Are Ostersund


#### In the script there is a connections variable which identify all the flight from a specific airport. So I extract first the departure location and then destination and I loop for each destination adding the departure to get all the route

In [337]:
columns=['from','to']
easyjet_route = pd.DataFrame( columns=columns)
from_airport=[]
to_airport=[]
for script in scripts:
    if script.text.find("Sitecore_WorldwideSettings")>0:
        script_text=script.text[int(script.text.find("Sitecore_RoutesData")):]
        #print(script_text)
        while script_text.find('"Connections":')>0:
        #for i in range(2):
            from_airport_text=script_text[int(script_text.find('"Name":'))+8:int(script_text.find('"},{"'))]
         #   if i==1:
          #      print(script_text)
            to_airport_text=script_text[int(script_text.find('"Connections":'))+15:int(script_text.find('B2bAreaCode'))-3]
            for airport in to_airport_text.split(','):
                if airport.find("*")<0:
                    to_airport.append(airport.replace('"',''))
                    from_airport.append(from_airport_text)
            script_text=script_text[int(script_text.find('},{"Connections":'))+2:]

In [338]:
len(from_airport)

2299

In [339]:
len(to_airport)

2299

In [340]:
for i in range(len(from_airport)):
    easyjet_route=easyjet_route.append({'from':from_airport[i],'to':to_airport[i]}, ignore_index=True)


In [342]:
easyjet_route.iloc[2289:]['from']='Zurich'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [347]:
easyjet_route=easyjet_route[easyjet_route['from'].str.find("All Airports")<1]

In [348]:
easyjet_route=easyjet_route.merge(mapping_iata, left_on="to",right_on="iata" ,how="inner")[["from","city"]].sort_values("from").reset_index(drop=True)

In [349]:
easyjet_route

,from,city
0,Aarhus,London Gatwick
1,Aarhus,Berlin Tegel
2,Aberdeen,London Gatwick
3,Aberdeen,Geneva
4,Aberdeen,London Luton
5,Agadir,London Gatwick
6,Agadir,Berlin Tegel
7,Agadir,Lyon
8,Agadir,Manchester
9,Alicante,Milan Malpensa


#### I took an example to see if for a specific flight I got the return

In [352]:
easyjet_route[(easyjet_route['from']=="Aarhus") | (easyjet_route['city']=="Aarhus")]

,from,city
0,Aarhus,London Gatwick
1,Aarhus,Berlin Tegel
268,Berlin Tegel,Aarhus
940,London Gatwick,Aarhus


#### I realized ryanair have all the route in one direction, so I add both ways

In [356]:
ryanair_route_return=ryanair_route.copy()

In [359]:
ryanair_route_return=ryanair_route_return.rename(columns={"from": "to", "to": "from"})


In [366]:
ryanair_route=ryanair_route.append(ryanair_route_return, ignore_index=True)
ryanair_route["from"]=ryanair_route["from"].str.replace(" ","")
ryanair_route["to"]=ryanair_route["to"].str.replace(" ","")

#### I join the two dataframe to see some common route, I say some because probably need a better mapping using iata code instead of city as they could use different naming.

In [404]:
some_common_route=ryanair_route.merge(easyjet_route, left_on=["from","to"],right_on=["from","city"],how="inner")

In [406]:
some_common_route=some_common_route[["from","to"]]

In [408]:
some_common_route.drop_duplicates()

,from,to
0,Agadir,Manchester
2,Alicante,Bristol
4,Alicante,Edinburgh
6,Alicante,Glasgow
8,Alicante,Liverpool
10,Alicante,Manchester
12,Amsterdam,Malaga
14,Barcelona,Bristol
16,Barcelona,Liverpool
18,Barcelona,Manchester
